In [1]:
!python -V

Python 3.9.16


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1685294200498, experiment_id='1', last_update_time=1685294200498, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    #df = pd.read_csv(filename)
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
#df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
#df_val = read_dataframe('./data/green_tripdata_2021-02.csv')
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.75871520516201

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ryan")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [12]:
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [24]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[13:27:38] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                                                     
[0]	validation-rmse:20.12474                                                                                                                                                               
[1]	validation-rmse:19.12686                                                                                                                                                               
[2]	validation-rmse:18.19449                                                                                                                                                               
[3]	validation-rmse:17.32429                                                                                                                                                               
[4]	validation-rmse:16.51178                                

[85]	validation-rmse:6.46959                                                                                                                                                               
[86]	validation-rmse:6.46793                                                                                                                                                               
[87]	validation-rmse:6.46635                                                                                                                                                               
[88]	validation-rmse:6.46484                                                                                                                                                               
[89]	validation-rmse:6.46334                                                                                                                                                               
[90]	validation-rmse:6.46212                                

[171]	validation-rmse:6.41132                                                                                                                                                              
[172]	validation-rmse:6.41091                                                                                                                                                              
[173]	validation-rmse:6.41063                                                                                                                                                              
[174]	validation-rmse:6.41015                                                                                                                                                              
[175]	validation-rmse:6.40991                                                                                                                                                              
[176]	validation-rmse:6.40963                               

[257]	validation-rmse:6.38742                                                                                                                                                              
[258]	validation-rmse:6.38726                                                                                                                                                              
[259]	validation-rmse:6.38699                                                                                                                                                              
[260]	validation-rmse:6.38681                                                                                                                                                              
[261]	validation-rmse:6.38660                                                                                                                                                              
[262]	validation-rmse:6.38627                               

[343]	validation-rmse:6.37148                                                                                                                                                              
[344]	validation-rmse:6.37130                                                                                                                                                              
[345]	validation-rmse:6.37110                                                                                                                                                              
[346]	validation-rmse:6.37089                                                                                                                                                              
[347]	validation-rmse:6.37070                                                                                                                                                              
[348]	validation-rmse:6.37059                               

...

[725]	validation-rmse:6.41930                                                                                                                                                              
[726]	validation-rmse:6.41921                                                                                                                                                              
[727]	validation-rmse:6.41920                                                                                                                                                              
[728]	validation-rmse:6.41901                                                                                                                                                              
[729]	validation-rmse:6.41887                                                                                                                                                              
[730]	validation-rmse:6.41883                               

[811]	validation-rmse:6.41084                                                                                                                                                              
[812]	validation-rmse:6.41079                                                                                                                                                              
[813]	validation-rmse:6.41063                                                                                                                                                              
[814]	validation-rmse:6.41078                                                                                                                                                              
[815]	validation-rmse:6.41065                                                                                                                                                              
[816]	validation-rmse:6.41056                               

[897]	validation-rmse:6.40470                                                                                                                                                              
[898]	validation-rmse:6.40455                                                                                                                                                              
[899]	validation-rmse:6.40428                                                                                                                                                              
[900]	validation-rmse:6.40408                                                                                                                                                              
[901]	validation-rmse:6.40398                                                                                                                                                              
[902]	validation-rmse:6.40405                               

[983]	validation-rmse:6.39737                                                                                                                                                              
[984]	validation-rmse:6.39747                                                                                                                                                              
[985]	validation-rmse:6.39740                                                                                                                                                              
[986]	validation-rmse:6.39743                                                                                                                                                              
[987]	validation-rmse:6.39729                                                                                                                                                              
[988]	validation-rmse:6.39729                               

[68]	validation-rmse:6.72102                                                                                                                                                               
 30%|███████████████████████████████████▍                                                                                  | 15/50 [11:57<18:18, 31.38s/trial, best loss: 6.28608183739523]

In [21]:
params = {
    'learning_rate': 0.38681283879104617,
    'max_depth': 18,
    'min_child_weight': 1.148588628778374,
    'objective': 'reg:linear',
    'reg_alpha': 0.1422529043450288,
    'reg_lambda': 0.3540021019352697,
    'seed': 42
}

mlflow.xgboost.autolog(disable=True)

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

[14:04:16] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:14.44297
[1]	validation-rmse:10.64911
[2]	validation-rmse:8.63994
[3]	validation-rmse:7.62500
[4]	validation-rmse:7.11991
[5]	validation-rmse:6.86368
[6]	validation-rmse:6.72631
[7]	validation-rmse:6.64875
[8]	validation-rmse:6.60663
[9]	validation-rmse:6.57980
[10]	validation-rmse:6.56315
[11]	validation-rmse:6.55417
[12]	validation-rmse:6.54858
[13]	validation-rmse:6.54344
[14]	validation-rmse:6.53832
[15]	validation-rmse:6.53368
[16]	validation-rmse:6.52925
[17]	validation-rmse:6.52631
[18]	validation-rmse:6.52347
[19]	validation-rmse:6.52053
[20]	validation-rmse:6.51850
[21]	validation-rmse:6.51665
[22]	validation-rmse:6.51294
[23]	validation-rmse:6.51096
[24]	validation-rmse:6.50650
[25]	validation-rmse:6.50436
[26]	validation-rmse:6.50098
[27]	validation-rmse:6.49975
[28]	validation-rmse:6.49877
[29]	validation-rmse:6.49704
[30]	validation-rms

[273]	validation-rmse:6.31800
[274]	validation-rmse:6.31780
[275]	validation-rmse:6.31753
[276]	validation-rmse:6.31689
[277]	validation-rmse:6.31636
[278]	validation-rmse:6.31566
[279]	validation-rmse:6.31559
[280]	validation-rmse:6.31601
[281]	validation-rmse:6.31579
[282]	validation-rmse:6.31576
[283]	validation-rmse:6.31555
[284]	validation-rmse:6.31540
[285]	validation-rmse:6.31544
[286]	validation-rmse:6.31526
[287]	validation-rmse:6.31510
[288]	validation-rmse:6.31481
[289]	validation-rmse:6.31450
[290]	validation-rmse:6.31409
[291]	validation-rmse:6.31368
[292]	validation-rmse:6.31347
[293]	validation-rmse:6.31341
[294]	validation-rmse:6.31295
[295]	validation-rmse:6.31258
[296]	validation-rmse:6.31219
[297]	validation-rmse:6.31180
[298]	validation-rmse:6.31158
[299]	validation-rmse:6.31191
[300]	validation-rmse:6.31113
[301]	validation-rmse:6.31140
[302]	validation-rmse:6.31147
[303]	validation-rmse:6.31068
[304]	validation-rmse:6.30956
[305]	validation-rmse:6.30927
[306]	vali

[547]	validation-rmse:6.28541
[548]	validation-rmse:6.28550
[549]	validation-rmse:6.28549
[550]	validation-rmse:6.28541
[551]	validation-rmse:6.28540
[552]	validation-rmse:6.28553
[553]	validation-rmse:6.28549
[554]	validation-rmse:6.28525
[555]	validation-rmse:6.28529
[556]	validation-rmse:6.28523
[557]	validation-rmse:6.28528
[558]	validation-rmse:6.28520
[559]	validation-rmse:6.28536
[560]	validation-rmse:6.28569
[561]	validation-rmse:6.28622
[562]	validation-rmse:6.28608
[563]	validation-rmse:6.28614
[564]	validation-rmse:6.28580
[565]	validation-rmse:6.28555
[566]	validation-rmse:6.28535
[567]	validation-rmse:6.28499
[568]	validation-rmse:6.28523
[569]	validation-rmse:6.28550
[570]	validation-rmse:6.28543
[571]	validation-rmse:6.28560
[572]	validation-rmse:6.28560
[573]	validation-rmse:6.28568
[574]	validation-rmse:6.28503
[575]	validation-rmse:6.28544
[576]	validation-rmse:6.28553
[577]	validation-rmse:6.28528
[578]	validation-rmse:6.28513
[579]	validation-rmse:6.28492
[580]	vali

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    params = {
        'learning_rate': 0.38681283879104617
        'max_depth': 18
        'min_child_weight': 1.148588628778374
        'objective': 'reg:linear'
        'reg_alpha': 0.1422529043450288
        'reg_lambda': 0.3540021019352697
        'seed': 42
    }

#     best_params = {
#         'learning_rate': 0.09585355369315604,
#         'max_depth': 30,
#         'min_child_weight': 1.060597050922164,
#         'objective': 'reg:linear',
#         'reg_alpha': 0.018060244040060163,
#         'reg_lambda': 0.011658731377413597,
#         'seed': 42
#     }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

## How to save a model as artifact in MLFlow

In [23]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ryan")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [29]:
mlflow.xgboost.autolog(disable=True)

In [30]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    params = {
        "learning_rate":	0.06971056201754594,
        "max_depth":	34,
        "min_child_weight":	1.1197956630070371,
        "objective":	'reg:linear',
        "reg_alpha":	0.03431106255339526,
        "reg_lambda":	0.271221458094817,
        "seed":	42
    }

    mlflow.log_params(params)   
    

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[09:31:57] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.94419
[1]	validation-rmse:18.79114
[2]	validation-rmse:17.72790
[3]	validation-rmse:16.74699
[4]	validation-rmse:15.84302
[5]	validation-rmse:15.01140
[6]	validation-rmse:14.24772
[7]	validation-rmse:13.54824
[8]	validation-rmse:12.90516
[9]	validation-rmse:12.31656
[10]	validation-rmse:11.77948
[11]	validation-rmse:11.28843
[12]	validation-rmse:10.84183
[13]	validation-rmse:10.43254
[14]	validation-rmse:10.06141
[15]	validation-rmse:9.72320
[16]	validation-rmse:9.41796
[17]	validation-rmse:9.13901
[18]	validation-rmse:8.88692
[19]	validation-rmse:8.65853
[20]	validation-rmse:8.45241
[21]	validation-rmse:8.26639
[22]	validation-rmse:8.09758
[23]	validation-rmse:7.94574
[24]	validation-rmse:7.80855
[25]	validation-rmse:7.68462
[26]	validation-rmse:7.57362
[27]	validation-rmse:7.47295
[28]	validation-rmse:7.38241
[29]	validation-rmse:7.29996
[30]	v

[273]	validation-rmse:6.38629
[274]	validation-rmse:6.38607
[275]	validation-rmse:6.38589
[276]	validation-rmse:6.38584
[277]	validation-rmse:6.38572
[278]	validation-rmse:6.38535
[279]	validation-rmse:6.38514
[280]	validation-rmse:6.38462
[281]	validation-rmse:6.38440
[282]	validation-rmse:6.38417
[283]	validation-rmse:6.38395
[284]	validation-rmse:6.38377
[285]	validation-rmse:6.38347
[286]	validation-rmse:6.38335
[287]	validation-rmse:6.38330
[288]	validation-rmse:6.38304
[289]	validation-rmse:6.38284
[290]	validation-rmse:6.38253
[291]	validation-rmse:6.38223
[292]	validation-rmse:6.38188
[293]	validation-rmse:6.38161
[294]	validation-rmse:6.38131
[295]	validation-rmse:6.38114
[296]	validation-rmse:6.38099
[297]	validation-rmse:6.38079
[298]	validation-rmse:6.38052
[299]	validation-rmse:6.38018
[300]	validation-rmse:6.37996
[301]	validation-rmse:6.37987
[302]	validation-rmse:6.37960
[303]	validation-rmse:6.37930
[304]	validation-rmse:6.37899
[305]	validation-rmse:6.37877
[306]	vali

[547]	validation-rmse:6.33764
[548]	validation-rmse:6.33751
[549]	validation-rmse:6.33743
[550]	validation-rmse:6.33737
[551]	validation-rmse:6.33731
[552]	validation-rmse:6.33737
[553]	validation-rmse:6.33715
[554]	validation-rmse:6.33700
[555]	validation-rmse:6.33684
[556]	validation-rmse:6.33665
[557]	validation-rmse:6.33655
[558]	validation-rmse:6.33642
[559]	validation-rmse:6.33625
[560]	validation-rmse:6.33595
[561]	validation-rmse:6.33588
[562]	validation-rmse:6.33573
[563]	validation-rmse:6.33561
[564]	validation-rmse:6.33547
[565]	validation-rmse:6.33525
[566]	validation-rmse:6.33516
[567]	validation-rmse:6.33511
[568]	validation-rmse:6.33499
[569]	validation-rmse:6.33485
[570]	validation-rmse:6.33477
[571]	validation-rmse:6.33476
[572]	validation-rmse:6.33460
[573]	validation-rmse:6.33455
[574]	validation-rmse:6.33436
[575]	validation-rmse:6.33437
[576]	validation-rmse:6.33418
[577]	validation-rmse:6.33405
[578]	validation-rmse:6.33390
[579]	validation-rmse:6.33370
[580]	vali

[821]	validation-rmse:6.31103
[822]	validation-rmse:6.31104
[823]	validation-rmse:6.31105
[824]	validation-rmse:6.31109
[825]	validation-rmse:6.31105
[826]	validation-rmse:6.31099
[827]	validation-rmse:6.31097
[828]	validation-rmse:6.31090
[829]	validation-rmse:6.31080
[830]	validation-rmse:6.31073
[831]	validation-rmse:6.31063
[832]	validation-rmse:6.31054
[833]	validation-rmse:6.31045
[834]	validation-rmse:6.31032
[835]	validation-rmse:6.31022
[836]	validation-rmse:6.31011
[837]	validation-rmse:6.31002
[838]	validation-rmse:6.31002
[839]	validation-rmse:6.30991
[840]	validation-rmse:6.30975
[841]	validation-rmse:6.30972
[842]	validation-rmse:6.30965
[843]	validation-rmse:6.30963
[844]	validation-rmse:6.30944
[845]	validation-rmse:6.30937
[846]	validation-rmse:6.30943
[847]	validation-rmse:6.30930
[848]	validation-rmse:6.30925
[849]	validation-rmse:6.30925
[850]	validation-rmse:6.30915
[851]	validation-rmse:6.30902
[852]	validation-rmse:6.30893
[853]	validation-rmse:6.30887
[854]	vali

## Can access model using Python function

In [31]:
logged_model = 'runs:/12fb0dac0f7b46f1beb257c1474eeb2f/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2023/05/29 09:38:39 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[09:38:39] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [33]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 12fb0dac0f7b46f1beb257c1474eeb2f

## Can access model using XGBoost

In [34]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[09:41:49] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [35]:
xgboost_model

In [36]:
y_pred = xgboost_model.predict(valid)

In [37]:
y_pred[:10]

array([14.805157 ,  7.1672354, 15.882237 , 24.255116 ,  9.242425 ,
       17.188751 , 11.746371 ,  8.733703 ,  8.982024 , 20.485725 ],
      dtype=float32)

In [ ]:
# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data))

In [38]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/jellyfish/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
